# Select the tract-patch with the higest number of visits and Show the Coadds and Fetch the sources in that Patch

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- member : DESC, rubin-inkind
- creation date : 2025-05-31
- last update : 2025-06-01

In [ ]:
import lsst.pipe.base
print(lsst.pipe.base.__version__)

In [ ]:
import sys
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
from lsst.geom import SpherePoint, degrees
from lsst.afw.image import ExposureF
from lsst.skymap import PatchInfo, Index2D
import numpy as np
import pandas as pd
import lsst.afw.display as afwDisplay
# %matplotlib widget

%matplotlib widget

In [ ]:
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time

In [ ]:
import seaborn as sns

In [ ]:
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.labelsize"] = "x-large"
plt.rcParams["axes.titlesize"] = "x-large"
plt.rcParams["xtick.labelsize"] = "x-large"
plt.rcParams["ytick.labelsize"] = "x-large"

# Set up some plotting defaults:
plt.rcParams.update({'figure.figsize' : (12, 8)})
plt.rcParams.update({'font.size' : 20})
plt.rcParams.update({'axes.linewidth' : 3})
plt.rcParams.update({'axes.labelweight' : 3})
plt.rcParams.update({'axes.titleweight' : 5})
plt.rcParams.update({'ytick.major.width' : 3})
plt.rcParams.update({'ytick.minor.width' : 2})
plt.rcParams.update({'ytick.major.size' : 8})
plt.rcParams.update({'ytick.minor.size' : 5})
plt.rcParams.update({'xtick.major.size' : 8})
plt.rcParams.update({'xtick.minor.size' : 5})
plt.rcParams.update({'xtick.major.width' : 3})
plt.rcParams.update({'xtick.minor.width' : 2})
plt.rcParams.update({'xtick.direction' : 'in'})
plt.rcParams.update({'ytick.direction' : 'in'})


In [ ]:
all_bands = ["u", "g", "r", "i", "z", "y"]
all_bands_colors = ["blue", "green", "red", "orange", "yellow", "purple"]

In [ ]:
import traceback

In [ ]:
# Define butler
from lsst.daf.butler import Butler

In [ ]:
!eups list lsst_distrib

In [ ]:
def nJy_to_ab_mag(f_njy):
    """Convert scalar or array flux in nJy to AB magnitude."""
    f_njy = np.asarray(f_njy)
    mag = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = f_njy > 0
    mag[mask] = -2.5 * np.log10(f_njy[mask]) + 31.4
    return mag


def nJy_err_to_ab_err(f_njy, f_err):
    """Propagate flux error to magnitude error."""
    f_njy = np.asarray(f_njy)
    f_err = np.asarray(f_err)
    mag_err = np.full_like(f_njy, fill_value=np.nan, dtype=float)
    mask = (f_njy > 0) & (f_err > 0)
    mag_err[mask] = (2.5 / np.log(10)) * (f_err[mask] / f_njy[mask])
    return mag_err


In [ ]:
def RetrieveDRPSources_forTarget(butler,center_coord,datasettype,where_clause,radius_cut=50):
    """
    Find the closest Sourcesthe target_coord 

    parameters:
    - butler
    - the coordinate of the target (center of the cone seach)
    - the datasettype name for the DRP object
    - where_clause : which contrain requirements on the tract and patch numbers
    - cut on angluar separation for the returned for the returned object

    Return
    - object Id with minimum separation , 
    - minimum separation (arcec),
    - the table of DRP objects within the radius_cut
    """

    ra_columns = ['u_ra', 'g_ra', 'r_ra', 'i_ra', 'z_ra', 'y_ra']
    dec_columns = ['u_dec', 'g_dec', 'r_dec', 'i_dec', 'z_dec', 'y_dec']

    
    therefs = butler.registry.queryDatasets(datasettype,  collections=collection, where=where_clause)

    for count,ref in enumerate(therefs):
        the_id = ref.dataId
        the_tract_id = the_id["tract"] 
        print(the_id)
        
        # catalog of rubin objects (a pandas Dataframe) inside the tract
        catalog = butler.get(ref)
        catalog = catalog[catalog["patch"] == patchNbSel] 
       
    
        nobjects = len(catalog)


        # Calcul de la moyenne ligne par ligne, en ignorant les NaN
        catalog['ra'] = catalog[ra_columns].mean(axis=1, skipna=True)
        catalog['dec'] = catalog[dec_columns].mean(axis=1, skipna=True)


        # extract the (ra,dec) coordinates for all te objects of the rubin-catalog
        ra_cat = catalog["ra"].values
        dec_cat = catalog["dec"].values
        # coordinates for all rubin-catalog points
        catalog_coords = SkyCoord(ra=ra_cat*u.deg, dec=dec_cat*u.deg)

        # Angular distance to target
        distances_arcsec = center_coord.separation(catalog_coords).arcsecond

        # add the separation angle to the ctalog
        catalog["sep"] = distances_arcsec


        # closest object from the target
        sepMin = distances_arcsec.min() 
        sepMin_idx = np.where(distances_arcsec == sepMin)[0][0]
    
        closest_obj = catalog[catalog["sep"] <=  sepMin]
                   
        # select a few of these sources to debug the closest candidate
        nearby_obj = catalog[distances_arcsec < radius_cut]
        
        return closest_obj, sepMin, nearby_obj


In [ ]:
def FetchTimesForVisits(visit_list):
    """
    """
    # On interroge la table visitDefinition
    Nvisit = len(visit_list)
        
    if Nvisit == 1:
        thevisit = visit_list.values[0]
        rows = registry.queryDimensionRecords("visit", where=f"visit={thevisit}")
    else:
        rows = registry.queryDimensionRecords("visit", where=f"visit in {tuple(visit_list)}")

    # 4. Construire un tableau des résultats
    results = []
    for row in rows:
        visit_id = row.id
        visit_airmass = 1./np.cos(Angle(row.zenith_angle,u.degree).rad)
        visit_azimuth = row.azimuth

        # Extraire l'instant de début de l'observation (Time astropy)
        start_time = row.timespan.begin

        # Convertir en MJD et ISO
        mjd = start_time.to_value("mjd")  # Ex: 60384.28718
        isot = start_time.to_value("isot")  # Ex: '2024-04-19 06:53:32.000'
    
        #mjd = row.startDate.toMjd()
        #utc = Time(mjd, format='mjd', scale='utc').to_value('iso')
        #results.append({"visit": visit_id, "mjd": mjd, "isot": isot})
        results.append({"visit": visit_id, "mjd": mjd, "isot": isot,"airmass":visit_airmass,"azimuth":visit_azimuth})

    df_times = pd.DataFrame(results).sort_values("visit")
    df_times.set_index("visit",inplace=True)
    return df_times

## RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

## Configuration

### Choose instrument

In [ ]:
# instrument = "LSSTCam"
instrument = "LSSTComCam"
# instrument = "LATISS"

### For LSSTCam : RubinTV, Campaigns , quicklook
- RubinTV : https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam
- https://rubinobs.atlassian.net/wiki/spaces/LSSTCOM/pages/467370016/LSSTCam+Commissioning+Planning
- LSSTCam DM campaign : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.2.-LSSTCam-Nightly-Validation-Pipeline
- Check campaign also here  https://rubinobs.atlassian.net/wiki/pages/diffpagesbyversion.action?pageId=48834013&selectedPageVersions=145%2C143
- fov-quicklook : https://usdf-rsp-dev.slac.stanford.edu/fov-quicklook/

### For LSSTComCam check here : 
- - Check here the collection available : https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs

In [ ]:
if instrument == "LSSTCam":
    repo = "/repo/embargo"
    instrument = "LSSTCam"
    collection_validation = instrument + "/runs/nightlyValidation"
    # collection_quicklook   = instrument + '/runs/quickLookTesting'
    collection_validation = os.path.join(collection_validation, "20250416/d_2025_04_15/DM-50157")
    date_start = 20250415
    date_selection = 20250416
    where_clause = "instrument = '" + f"{instrument}" + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"
    skymapName = "lsst_cells_v1"

elif instrument == "LSSTComCam":
    repo = "/repo/main"
    collection_validation = "LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359"  # work
    date_start = 20241024
    date_selection = 20241211
    skymapName = "lsst_cells_v1"
    where_clause = "instrument = \'" + instrument+ "\'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063
    PATCHSEL = 14

elif instrument == "LSSTComCamSim":
    repo = "/repo/main"
    collection_validation = "LSSTComCamSim/*"  # work
    date_start = 20241024
    date_selection = 20241211
    skymapName = "ops_rehersal_prep_2k_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

elif instrument == "LATISS":
    repo = "/repo/main"
    # collection_validation = instrument + "/runs/quickLook"
    collection_validation = instrument + "/raw/all"
    date_start = 20221001
    date_selection = 20221001
    skymapName = "latiss_v1"
    where_clause = "instrument = '" + instrument + "'"
    where_clause_date = where_clause + f"and day_obs >= {date_start}"

    NDET = 9
    TRACTSEL = 5063

In [ ]:
collectionStr = collection_validation.replace("/", "_")

## Access to Butler registry

In [ ]:
# Initialize the butler repo:
butler = Butler(repo, collections=collection_validation)
registry = butler.registry

## Create a skymap object

In [ ]:
# skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)

In [ ]:
try:
    skymap = butler.get("skyMap", skymap=skymapName, collections=collection_validation)
except Exception as inst:
    print(type(inst))  # the exception type
    print(inst.args)  # arguments stored in .args
    print(inst)  # __str__ allows args to be printed directly,

In [ ]:
camera = butler.get("camera", collections=collection_validation, instrument=instrument)

## Dump registry into a pandas dataframe

- Faster method to decode the registry in pandas dataframe : first save deconded filed into a list of fields and then flush the whole list in pandas instead of row by row
- Be carefull the registry variable change in name and type perhaps depending on DM_version

In [ ]:
print(where_clause_date)

In [ ]:
columns = [
    "id",
    "obs_id",
    "day_obs",
    "seq_num",
    "time_start",
    "time_end",
    "type",
    "target",
    "filter",
    "zenith_angle",
    "expos",
    "ra",
    "dec",
    "skyangle",
    "azimuth",
    "zenith",
    "science_program",
    "jd",
    "mjd",
]

In [ ]:
df_exposure = pd.DataFrame(
    {
        "id": pd.Series(dtype="int"),
        "obs_id": pd.Series(dtype="int"),
        "day_obs": pd.Series(dtype="int"),
        "seq_num": pd.Series(dtype="int"),
        "time_start": pd.Series(dtype="str"),  # ou 'datetime64[ns]' si c’est un datetime
        "time_end": pd.Series(dtype="str"),  # idem
        "type": pd.Series(dtype="str"),
        "target": pd.Series(dtype="str"),
        "filter": pd.Series(dtype="str"),
        "zenith_angle": pd.Series(dtype="float"),
        "expos": pd.Series(dtype="float"),  # ou 'int' selon le cas
        "ra": pd.Series(dtype="float"),
        "dec": pd.Series(dtype="float"),
        "skyangle": pd.Series(dtype="float"),
        "azimuth": pd.Series(dtype="float"),
        "zenith": pd.Series(dtype="float"),
        "science_program": pd.Series(dtype="str"),
        "jd": pd.Series(dtype="float"),
        "mjd": pd.Series(dtype="float"),
    }
)

In [ ]:
# save the data array in rows before saving in pandas dataframe
rows = []
for count, info in enumerate(registry.queryDimensionRecords("exposure", where=where_clause_date)):
    try:
        jd_start = info.timespan.begin.value
        jd_end = info.timespan.end.value
        the_Time_start = Time(jd_start, format="jd", scale="utc")
        the_Time_end = Time(jd_end, format="jd", scale="utc")
        mjd_start = the_Time_start.mjd
        mjd_end = the_Time_end.mjd
        isot_start = the_Time_start.isot
        isot_end = the_Time_end.isot

        if count == 0:
            print("===== Time Conversion Debug Info =====")
            print(f"JD start      : {jd_start} (type: {type(jd_start)})")
            print(f"JD end        : {jd_end} (type: {type(jd_end)})")
            print(f"MJD start     : {mjd_start} (type: {type(mjd_start)})")
            print(f"MJD end       : {mjd_end} (type: {type(mjd_end)})")
            print(f"ISOT start    : {isot_start} (type: {type(isot_start)})")
            print(f"ISOT end      : {isot_end} (type: {type(isot_end)})")
            print("=======================================")

        # put row in a dictionnary before stacking
        row = {
            "id": info.id,
            "obs_id": info.obs_id,
            "day_obs": info.day_obs,
            "seq_num": info.seq_num,
            "time_start": isot_start,
            "time_end": isot_end,
            "type": info.observation_type,
            "target": info.target_name,
            "filter": info.physical_filter,
            "zenith_angle": info.zenith_angle,
            "expos": info.exposure_time,  # Exemple : adapter selon ton objet
            "ra": info.tracking_ra,
            "dec": info.tracking_dec,
            "skyangle": info.sky_angle,
            "azimuth": info.azimuth,
            "zenith": info.zenith_angle,
            "science_program": info.science_program,
            "jd": float(jd_start),
            "mjd": float(mjd_start),
        }
        rows.append(row)

    except ValueError as e:
        print(f"Erreur de valeur : {e}")
    except FileNotFoundError as e:
        print(f"Fichier introuvable : {e}")
    except Exception as e:
        print(f"Erreur inattendue : {type(e).__name__} - {e}")
        print(f">>>   Unexpected error at row {count}:", sys.exc_info()[0])
        traceback.print_exc()  # affiche la stack trace complète

In [ ]:
# Création finale du DataFrame
df_exposure = pd.DataFrame(rows)

In [ ]:
df_exposure

In [ ]:
# df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

## Select science exposures

In [ ]:
df_science = df_exposure[df_exposure.type == "science"]
df_science.reset_index(drop=True, inplace=True)

## Add Tract-Patches

In [ ]:
df=df_science.copy()
df["band"] = df["filter"].apply(lambda x : x.split("_")[0])

In [ ]:
#df_with_tract_patch = add_tract_patch(df, butler)

In [ ]:
def get_tract_patch(row, skymap):
    if pd.isna(row['ra']) or pd.isna(row['dec']):
        return pd.Series({"tract": None, "patch": None})
    
    target_point = SpherePoint(row['ra'], row['dec'], degrees)

    tract_info = skymap.findTract(target_point)
    patch_info = tract_info.findPatch(target_point)
    tractNbSel = tract_info.getId()
    patchNbSel =  patch_info.getSequentialIndex()
    patch_index_str = f"{patch_info.getIndex()[0]},{patch_info.getIndex()[1]}"
   
    
    return pd.Series({"tract":   tractNbSel, "patch":  patchNbSel, "patch_str": patch_index_str})


In [ ]:
df = df.copy()
df[['tract', 'patch', "patch_str"]] = df.apply(get_tract_patch, axis=1, args=(skymap,))

In [ ]:
df

## Plots histograms with the number of visits and time sequences

### Visits grouped by tract and target field

In [ ]:
# 1. Trier le DataFrame par numéro de tract
df = df.sort_values("tract")
# 2. Créer la colonne tag après le tri
df["tag"] = df["tract"].astype(str) + "_" + df["target"]
# 3. Grouper par tag et band, et compter
grouped_tag = df.groupby(['tag', 'band']).size().reset_index(name='count')
# 4. Définir l'ordre des tags selon l'ordre dans df trié
tag_order = df["tag"].drop_duplicates().tolist()

In [ ]:
# Forcer l'ordre des bandes
band_order = ['u', 'g', 'r', 'i', 'z', 'y']
color_map = {
    'u': 'blue',
    'g': 'green',
    'r': 'red',
    'i': 'orange',
    'z': 'gray',
    'y': 'black'
}

In [ ]:
# 6. Tracer le barplot
plt.figure(figsize=(20, 8))
sns.barplot(
    data=grouped_tag,
    x='tag',
    y='count',
    hue='band',
    hue_order=band_order,
    palette=color_map,
    order=tag_order  # ordre des tags triés par tract
)

plt.xlabel("Visited fields / tract")
plt.ylabel("Number of visits per band")
plt.title("Number of visits per band in each tract",fontsize=20,fontweight="bold")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Band")
plt.tight_layout()
plt.show()

In [ ]:
# 6. Tracer le barplot
plt.figure(figsize=(8, 20))
sns.barplot(
    data=grouped_tag,
    x='count',
    y='tag',
    hue='band',
    hue_order=band_order,
    palette=color_map,
    order=tag_order  # ordre des tags triés par tract
)

plt.ylabel("Visited fields / tract")
plt.xlabel("Number of visits per band")
plt.title("Number of visits per band in each tract",fontsize=20,fontweight="bold")
#plt.xticks(rotation=45, ha='right')
plt.legend(title="Band")
plt.tight_layout()
plt.show()

### Visits detailed by tract and patches

In [ ]:
# Grouper et compter
grouped = df.groupby(['tract', 'patch', 'band']).size().reset_index(name='count')

In [ ]:
#grouped['tract_patch'] = grouped['tract'].astype(str) + '_' + grouped['patch_str']
grouped['tract_patch'] = grouped['tract'].astype(str) + '_' + grouped['patch'].astype(str)

In [ ]:
plt.figure(figsize=(18, 6))
sns.barplot(
    data=grouped,
    x='tract_patch',
    y='count',
    hue='band',
    hue_order=band_order,
    palette=color_map
)

plt.xlabel("Tract_Patch")
plt.ylabel("Number of visits per bands")
plt.title("Number of visits per bands in each (tract, patch)")
plt.xticks(rotation=45, ha='right')
plt.legend(title="Band")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, max(6, len(grouped['tract_patch'].unique()) * 0.8)))  # Adapter la hauteur

sns.barplot(
    data=grouped,
    y='tract_patch',  # axe Y = les (tract, patch)
    x='count',        # axe X = nombre de visites
    hue='band',
    hue_order=band_order,
    palette=color_map
)

plt.ylabel("Tract_Patch")
plt.xlabel("Number of visits per bands")
plt.title("Number of visits per bands in field of view (tract, patch)")
plt.legend(title="Band", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.tight_layout()
plt.show()


## Which (tract,patch) has the maximum number of visits in y band

In [ ]:
grouped

In [ ]:
# Filtrer la bande 'y'
df_y = df[df['band'] == 'y']

# Grouper par tract et patch
grouped_y = df_y.groupby(['tract', 'patch']).size().reset_index(name='count')

# Trouver le max
max_row = grouped_y.loc[grouped_y['count'].idxmax()]


tractNbSel = max_row['tract']
patchNbSel = max_row['patch']


In [ ]:
print(f"Tract: {tractNbSel}, Patch: {patchNbSel}, Number of Visits in y band: {max_row['count']}")

## Time sequences of arrival time in selected tract-patch

In [ ]:
df_sel = df[(df['tract'] == tractNbSel) & (df['patch'] == patchNbSel)]

In [ ]:
plt.figure(figsize=(16, 8))

sns.stripplot(
    data=df_sel,
    x='mjd',
    y='band',
    hue='band',  # <- correction
    order=band_order,
    palette=color_map,
    size=10,
    alpha=0.7,
    dodge=False,
    legend=False  # <- évite de doubler les légendes
)

plt.xlabel("MJD")
plt.ylabel("Band")
plt.title(f"Sequence of visits in tract={tractNbSel}, patch={patchNbSel}")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


## Retrieve all Coadds

In [ ]:
all_dataIds = [
    {
        "band": band,
        "tract": tractNbSel,
        "patch": patchNbSel,
        "skymap": skymapName,
    }
    for band in all_bands
]

In [ ]:
all_deepCoadds = []
all_deepCoadds_wcs = []
all_titles = []
for idx, band in enumerate(all_bands):
    the_band = band
    the_dataId = all_dataIds[idx]
    the_title = f"({tractNbSel},{patchNbSel}) " + f" band {the_band}"
    try:
        coadd_exp = butler.get("deepCoadd_calexp", the_dataId)
        wcs = coadd_exp.getWcs()
        psf = coadd_exp.getPsf()
        all_deepCoadds.append(coadd_exp)
        all_deepCoadds_wcs.append(wcs)
        all_titles.append(the_title)

    except Exception as inst:
        print(f"{key} :: catch Exception for band {band}")
        print(type(inst))  # the exception type
        print(inst.args)  # arguments stored in .args
        print(inst)  # __str_

## Plot all Deep Coadds with Firefly

In [ ]:
if 0:
    afwDisplay.setDefaultBackend("firefly")
    N = len(all_deepCoadds)
    for count in range(N):
        band = all_bands[count]
        display = afwDisplay.Display(frame=count + 1)
        # cannot succeed to show white stars on dark sky
        display.setImageColormap("gray")
        display.scale("asinh", "zscale")
        display.mtv(all_deepCoadds[count].image, title=all_titles[count])

### Clean

In [ ]:
# display.clearViewer()

## Selecting objects for Light Curves

In [ ]:
FLAG_DUMP_COLLECTIONS = False
FLAG_DUMP_DATASETS = False
FLAG_DUMP_OBJECTSTABLECOLUMNS = False
FLAG_CUT_OBJECTSMAG = True
FLAG_CUT_OBJECTSSNR = True

In [ ]:
# Check here the collections available
if FLAG_DUMP_COLLECTIONS:
    for _ in sorted(registry.queryCollections(expression=instrument + "/*")):
        if "/calib/" not in _ and "u/" not in _:
            print(_)

In [ ]:
if FLAG_DUMP_DATASETS:
    for datasetType in registry.queryDatasetTypes():
        if registry.queryDatasets(datasetType, collections=collection_validation).any(
            execute=False, exact=False
        ):
            # Limit search results to the data products
            if (
                ("_config" not in datasetType.name)
                and ("_log" not in datasetType.name)
                and ("_metadata" not in datasetType.name)
                and ("_resource_usage" not in datasetType.name)
                and ("Plot" not in datasetType.name)
                and ("Metric" not in datasetType.name)
                and ("metric" not in datasetType.name)
                and (("Table" in datasetType.name) or ("Zeropointp" in datasetType.name) or ("fgcm" in datasetType.name) or ("transm" in datasetType.name) or ("Transm" in datasetType.name)
                or ("source" in datasetType.name) or ("Source" in datasetType.name) or ("object" in datasetType.name) or ("Object" in datasetType.name))
            ):
                
                print(datasetType)

In [ ]:
BANDSEL = "g"

In [ ]:
tract_info = skymap[tractNbSel]
patch_info = tract_info.getPatchInfo(patchNbSel)

In [ ]:
# Centre du patch en pixels dans le système de coordonnées du tract
center_pix = patch_info.getOuterBBox().getCenter()
# Transformation en coordonnées célestes (RA/Dec)
center_coord = tract_info.getWcs().pixelToSky(center_pix)
# Extraire RA et Dec (en degrés)
ra = center_coord.getRa().asDegrees()
dec = center_coord.getDec().asDegrees()
print(f"Patch {patchNbSel} in Tract {tractNbSel} : center RA: {ra:.6f}, Dec: {dec:.6f}")

In [ ]:
df_sel[["id","band"]]

In [ ]:
df_bandsel = df_sel[df_sel.band == BANDSEL]

In [ ]:
df_bandsel

In [ ]:
tuple_of_visits = tuple(df_bandsel.id)
tuple_of_visits 

## Search in Objects

### Access to objectTable
    DatasetType('objectTable', {skymap, tract, patch}, DataFrame

In [ ]:
datasettype = 'objectTable'
where_clause_objects = f"skymap = '{skymapName}' AND tract = {tractNbSel} AND patch = {patchNbSel}"
ra_columns = ['u_ra', 'g_ra', 'r_ra', 'i_ra', 'z_ra', 'y_ra']
dec_columns = ['u_dec', 'g_dec', 'r_dec', 'i_dec', 'z_dec', 'y_dec']

In [ ]:
therefs = butler.registry.queryDatasets(
    datasettype,
    where=where_clause_objects,
    collections = collection_validation,
)

listref = list(therefs)
nref = len(listref)
print(f"nref = {nref}")

In [ ]:
for count,ref in enumerate(therefs):
    the_id = ref.dataId
    the_tract_id = the_id["tract"] 
    print(the_id)
        
    # catalog of rubin objects (a pandas Dataframe) inside the tract
    catalog = butler.get(ref)
    catalog = catalog[catalog["patch"] == patchNbSel] 
       
    
    nobjects = len(catalog)


    # Calcul de la moyenne ligne par ligne, en ignorant les NaN
    catalog['ra'] = catalog[ra_columns].mean(axis=1, skipna=True)
    catalog['dec'] = catalog[dec_columns].mean(axis=1, skipna=True)

    # compute magnitude AB
    for ib,band in enumerate(all_bands):
        id_name = "objectId"
        id_parentname = "parentObjectId"
        x_name = f"{band}_centroid_x"
        y_name = f"{band}_centroid_y"
        coord_ra_name = "coord_ra"
        coord_dec_name = "coord_dec"
        ra_name = f"{band}_ra"
        dec_name = f"{band}_dec"
        decl_name = f"{band}_decl"
        raerr_name = f"{band}_raErr"
        decerr_name = f"{band}_decErr"
        extendedness_name = f"{band}_extendedness"
        blendness_name = f"{band}_blendedness"
        psfflux_name = f"{band}_psfFlux"
        psffluxerr_name = f"{band}_psfFluxErr"
        psfmag_name = f"{band}_psfMag"
        psfmagerr_name = f"{band}_psfMagErr"
        psfflux_free_name = f"{band}_free_psfFlux"
        if band == BANDSEL:
            psfmag_name_keep =  psfmag_name 
            psfmagerr_name_keep = psfmagerr_name 
            color_keep = all_bands_colors[ib]
            extendedness_name_keep = extendedness_name
            blendness_name_keep = blendness_name
            
        catalog[psfmag_name] = catalog[psfflux_name].apply(nJy_to_ab_mag)
        catalog[psfmagerr_name] = nJy_err_to_ab_err(catalog[psfflux_name], catalog[psffluxerr_name])

In [ ]:
catalog.dropna(subset=[psfmag_name_keep,psfmagerr_name_keep], inplace=True)
catalog

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,6),layout="constrained")
catalog.plot.scatter(x=psfmag_name_keep, y = psfmagerr_name_keep,c= color_keep,ax=ax1,ylim=(0.,0.5))
catalog[psfmag_name_keep].hist(bins=50,ax=ax2,color= color_keep, histtype="step",linewidth=3,label=psfmag_name_keep)
ax2.set_xlabel(psfmag_name_keep)
plt.suptitle(collectionStr)
plt.show()

In [ ]:
selection_cut = (catalog[psfmag_name_keep]<22) & (catalog[extendedness_name_keep] == 0) & (catalog[blendness_name_keep] ==0) 
catalog_object_sel = catalog[selection_cut]
catalog_object_sel.reset_index(drop=True,inplace=True)

In [ ]:
catalog_object_sel[["objectId","ra","dec",psfmag_name_keep]]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8),layout="constrained")
sc = catalog_object_sel.plot.scatter(x="ra",y="dec",c=psfmag_name_keep,colormap="jet_r",ax=ax,marker="o",s=50,colorbar=False)
ax.set_aspect("equal")
ax.invert_xaxis()
ax.set_title("objects position")
cbar = fig.colorbar(sc.collections[0], ax=ax, orientation="horizontal", shrink=0.7, pad=0.05)
cbar.set_label(psfmag_name_keep)

plt.show()

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(16,6),layout="constrained")
catalog_object_sel.plot.scatter(x=psfmag_name_keep, y = psfmagerr_name_keep,c= color_keep,ax=ax1)
catalog_object_sel[psfmag_name_keep].hist(bins=50,ax=ax2,color= color_keep, histtype="step",linewidth=3,label=psfmag_name_keep)
ax2.set_xlabel(psfmag_name_keep)
plt.suptitle(collectionStr)
plt.show()

In [ ]:
#list(catalog_cut.columns)

In [ ]:
if 1:
    afwDisplay.setDefaultBackend("firefly")
    N = len(all_deepCoadds)
    for count in range(N):
        band = all_bands[count]
        if band == BANDSEL:
            display = afwDisplay.Display(frame= 1)
            # cannot succeed to show white stars on dark sky
            display.setImageColormap("gray")
            display.scale("asinh", "zscale")
            display.mtv(all_deepCoadds[count].image, title=all_titles[count])
            wcs = all_deepCoadds_wcs[count]

    x_name = "ra"
    y_name = "dec"
    with display.Buffering():
        for index, row in catalog_object_sel.iterrows():

            ra_deg = row[x_name]
            dec_deg = row[y_name]
              
            sky_coord = SpherePoint(ra_deg * degrees, dec_deg * degrees) 
            pixel_coord = wcs.skyToPixel(sky_coord)

            xpix = pixel_coord.x
            ypix = pixel_coord.y
                
            display.dot("o", xpix, ypix, size=50, ctype=color_keep)
            display.dot(str(index), xpix+80, ypix+80, size=2, ctype=color_keep)

## Search for the Sources

### Access to sourceTable
    DatasetType('sourceTable', {band, instrument, day_obs, detector, physical_filter, visit}, DataFrame)

In [ ]:
datasettype = 'sourceTable'

In [ ]:
#dataId = {"band": BANDSEL, "instrument": instrument, "visit" : 2024111900097}

In [ ]:
#### Build the where clause
where_clause_sources = where_clause_date + f" and visit in {tuple_of_visits}"
where_clause_sources

In [ ]:
therefs = butler.registry.queryDatasets(
    datasettype,
    where=where_clause_sources,
    collections = collection_validation,
)

listref = list(therefs)
nref = len(listref)
print(f"nref = {nref}")

In [ ]:
all_src = []
for count,ref in enumerate(therefs):
    the_id = ref.dataId
    #print(the_id)
    srccatalog = butler.get(ref)
    all_src.append(srccatalog)
    

In [ ]:
t =  pd.concat(all_src)

In [ ]:
t = t[t.parentSourceId  == 0 ]	

In [ ]:
list(t.columns)

In [ ]:
sources_cut = (t['detect_isPrimary']) & (t["extendedness"] == 0 ) 
t_sel = t[sources_cut]

In [ ]:
t_sel[['ra','dec','visit','psfFlux','psfFluxErr','ap35Flux', 'localPhotoCalib','detect_isPrimary','extendedness','sky_source']]

In [ ]:
t_sel = t_sel.copy()
t_sel["dateobs"] = t_sel.apply(lambda x: x['visit']//100_000, axis=1)
t_sel["seq"] = t_sel["visit"] - t_sel["dateobs"]*100_000
t_sel["mags"] = t_sel["psfFlux"].apply(nJy_to_ab_mag)
t_sel["magerr"] = nJy_err_to_ab_err(t_sel["psfFlux"], t_sel["psfFluxErr"])                               
t_sel["valid"] = (t_sel["psfFlux"] >0) & (~t_sel["psfFlux_flag"])
visit_list = t_sel["visit"].astype(int)

In [ ]:
visit_list

In [ ]:
df_times = FetchTimesForVisits(visit_list)

In [ ]:
t_sel["mjd"] = t_sel["visit"].apply(lambda x: df_times.loc[x]["mjd"])
t_sel["isot"] = t_sel["visit"].apply(lambda x: df_times.loc[x]["isot"])
t_sel["airmass"] = t_sel["visit"].apply(lambda x: df_times.loc[x]["airmass"])
t_sel["azimuth"] = t_sel["visit"].apply(lambda x: df_times.loc[x]["azimuth"])

In [ ]:
t_sel

In [ ]:
catalog_sources_coords = SkyCoord(ra=t_sel["ra"].values*u.deg, dec=t_sel["dec"].values*u.deg)

In [ ]:
from matplotlib import gridspec

sep_cut = 5.0
for idx, row in catalog_object_sel.iterrows(): 
    t_src = t_sel.copy()
    target_object_ra = row["ra"]
    target_object_dec = row["dec"]
    title_object = f"{idx}) : {row['objectId']}"  # Attention aux guillemets imbriqués

    target_object_coords = SkyCoord(ra=target_object_ra*u.deg, dec=target_object_dec*u.deg)
    distances_arcsec = target_object_coords.separation(catalog_sources_coords).arcsecond
    t_src["sep"] = distances_arcsec
    sepmin = distances_arcsec.min()
    t_src_cut = t_src[t_src["sep"] < sep_cut]
    
    # Créer figure et GridSpec avec 2 colonnes
    fig = plt.figure(figsize=(16, 3))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 3], wspace=0.3)  # largeur relative ax1:ax2 = 1:2.5

    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])

    # Histogramme à gauche
    ax1.hist(distances_arcsec, bins=60, range=(0, 50),facecolor="b")
    ax1.set_title(title_object + f" sep = {sepmin:.2f}\"", fontsize=10)
    ax1.set_xlabel("angular sep (arcsec)",fontsize=12)

    # Tu peux dessiner autre chose à droite ici (scatter, image, etc.)
    #ax2.text(0.5, 0.5, "Contenu pour ax2", ha='center', va='center')
    #ax2.set_axis_off()
    ax2.errorbar(t_src_cut["mjd"].values, t_src_cut["mags"].values,yerr=t_src_cut["magerr"].values,color="r") 
    ax2.errorbar(
    t_src_cut["mjd"].values,
    t_src_cut["mags"].values,
    yerr=t_src_cut["magerr"].values,
    fmt="o",                      # 'o' pour des points
    markersize=4,                 # taille des points
    capsize=2,                    # petites barres horizontales aux extrémités
    elinewidth=1,                 # épaisseur des barres d'erreur
    ecolor="gray",                # couleur des barres d'erreur
    markerfacecolor="black",     # couleur intérieure des points
    markeredgecolor="black",     # contour des points
    alpha=0.8                     # transparence
    )

    ax2.set_title(f"Light Curves in band {BANDSEL}", fontsize=10)
    ax2.set_xlabel("MJD", fontsize=9)
    ax2.set_ylabel("Magnitude (mag)", fontsize=9)
    ax2.invert_yaxis()  # Plus lumineux = vers le haut (standard en astronomie)
    ax2.grid(True, linestyle="--", alpha=0.5)

    plt.tight_layout()
    plt.show()
